<a href="https://colab.research.google.com/github/hhoppe/advent_of_code_2020/blob/main/advent_of_code_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of code 2020

Jupyter [notebook](https://github.com/hhoppe/advent_of_code_2020/blob/main/advent_of_code_2020.ipynb) by [Hugues Hoppe](http://hhoppe.com/); December 2020.

I participated in the 25-day [Advent of Code](https://adventofcode.com/) for the first time this year, thanks to encouragement from colleagues, especially [Sascha Häberling](https://github.com/shaeberling).  It was great fun and provided a nice opportunity to learn more advanced Python.

In the event, many people compete to solve the puzzles as quickly as possible --- see the impressive times on the [leaderboard](https://adventofcode.com/2020/leaderboard).
My approach was much more casual, although I did aim to finish the puzzle each evening.

Later, I went back to explore more **polished and efficient solutions**.
Can the code be expressed more succinctly?
What is the fastest algorithm given the constraints of interpreted Python?
Along the way, I discovered the [`numba`](https://numba.pydata.org/) package which can JIT-compile native code to overcome bottlenecks;
is it practical for these problems?  Yes, it can help greatly!

This notebook is organized such that **each day is self-contained**, i.e., it can be run independently after the common preamble in the next section.
The preamble introduces just one handy function `check_eq()`, so the solution algorithms are easily recognizable.

**For each day**, the first notebook cell defines a `puzzle` object:

> ```
  puzzle = advent.puzzle(day=1)
```
The puzzle input string is automatically read into the attribute `puzzle.input`.
This input string is unique to each Advent participant.
By default, the notebook uses [my input data](https://github.com/hhoppe/advent_of_code_2020/tree/main/data) stored on GitHub,
but the variable `INPUT_DATA_PATH_OR_URL_FORMAT` can refer to any directory or URL.
Similarly, the (per-user) puzzle answers are read from my JSON [file](https://github.com/hhoppe/advent_of_code_2020/blob/main/data/answers.json) named in `ANSWERS_PATH_OR_URL`.
> 
> For each of the two puzzle parts, a function (e.g. `process1`) takes an input string and returns a string or integer answer.
Using calls like the following, we time the execution of the function and verify the answers:
```
  puzzle.verify(part=1, func=process1)
  puzzle.verify(part=2, func=process2)
```

At the end of the notebook, a table summarizes **timing results**.

Some **conclusions**:

- The Jupyter/IPython notebook is a convenient environment to assemble descriptions, notes, code, small test inputs, and results.
- The cloud-based CPU kernel/runtime provided by Colab works nicely.
- With the help of the `numba` library (for days 11, 15, and 23), all of this year's puzzles can be solved in 1 second or less.
- Remarkably, the total execution time across all 25 puzzles is approximately 4 s.
- Initially I stored puzzle inputs within the notebook, but this adds clutter and runs inefficiently.

## Common code

In [1]:
INPUT_DATA_PATH_OR_URL_FORMAT = 'https://raw.githubusercontent.com/hhoppe/advent_of_code_2020/main/data/input{:02d}.txt'
ANSWERS_PATH_OR_URL = 'https://raw.githubusercontent.com/hhoppe/advent_of_code_2020/main/data/answers.json'

In [2]:
import bisect
import collections
import contextlib
import dataclasses
import functools
import itertools
import json
import math
import operator
import pprint
import re
import sys
import textwrap
import time
from typing import Callable, Dict, Optional, Tuple, Union
import unittest.mock
import urllib.request

import numpy as np

In [3]:
try:
  import numba
  using_numba = True
  numba_njit = numba.njit
except ModuleNotFoundError:
  print('numba unavailable')
  using_numba = False
  def noop_decorator(*args, **kwargs):
    """Function decorator that ignores any arguments and does nothing."""
    return (args[0] if len(args) == 1 and callable(args[0]) and not kwargs
            else noop_decorator)
  numba_njit = noop_decorator

In [4]:
def check_eq(a, b):
  if np.any(a != b):
    raise AssertionError(f'{a!r} == {b!r}')


def read_contents(path_or_url: str) -> bytes:
  if path_or_url.startswith(('http://', 'https://')):
    with urllib.request.urlopen(path_or_url) as response:
      data: bytes = response.read()
    return data
  with open(path_or_url, 'rb') as f:
    return f.read()


@dataclasses.dataclass
class PuzzlePart:
  func: Callable[[str], Union[str, int]]
  answer: Union[str, int]
  slow: bool
  elapsed_time: float = -0.0

  def compute(self, s: str, silent=False, repeat=1) -> None:
    with contextlib.ExitStack() as stack:
      if silent:
        for f in ('sys.stdout', 'sys.stderr', 'IPython.display.display'):
          stack.enter_context(unittest.mock.patch(f))
      if self.slow and not advent.also_run_slow_tests:
        print('Skipping slow computation because also_run_slow_tests is False')
        return
      elapsed_times = []
      for _ in range(repeat):
        start_time = time.time()
        result = self.func(s)
        elapsed_times.append(time.time() - start_time)
        if result != self.answer:
          raise AssertionError(f'Result {result} != expected {self.answer}')
      self.elapsed_time = min(elapsed_times)
      print(f'(Time: {self.elapsed_time:#6.3f})', flush=True)


@dataclasses.dataclass
class Puzzle:
  day: int
  input: str = ''
  parts: Dict[int, PuzzlePart] = dataclasses.field(default_factory=dict)  # 1..2

  def __post_init__(self) -> None:
    if self.input:
      self.input = self.input.lstrip('\n')
    else:
      path_or_url = INPUT_DATA_PATH_OR_URL_FORMAT.format(self.day)
      self.input = read_contents(path_or_url).decode()
    advent.puzzles[self.day] = self
    self.print_summary()

  def print_summary(self) -> None:
    lines = [(line[:80] + ' ... ' + line[-35:] if len(line) > 120 else line)
            for line in self.input.splitlines()]
    lines2 = lines[:8] + [' ...'] + lines[-4:] if len(lines) > 13 else lines
    s = '\n'.join(lines2)
    print(f'For day {self.day}, puzzle.input has {len(lines)} lines:\n{s}')
    print(f'And the stored answers are: {advent.answers[self.day]}')

  def verify(self, part: int, func: Callable[[str], Union[str, int]],
            answer: Optional[Union[str, int]] = None, slow=False) -> None:
    assert part in (1, 2)
    if answer is None:
      answer = advent.answers[self.day].get(part)
    else:
      advent.answers[self.day][part] = answer
    self.parts[part] = puzzle_part = PuzzlePart(func, answer, slow)
    puzzle_part.compute(self.input)


@dataclasses.dataclass
class Advent:
  also_run_slow_tests: bool
  puzzles: Dict[int, Puzzle] = dataclasses.field(default_factory=dict)  # 1..25
  answers: Dict[int, Dict[int, Union[str, int]]] = dataclasses.field(
      default_factory=lambda: collections.defaultdict(dict))

  def __post_init__(self) -> None:
    if ANSWERS_PATH_OR_URL:
      def keys_to_int(d): return {int(k): v for k, v in d.items()}
      self.answers = json.loads(read_contents(ANSWERS_PATH_OR_URL).decode(),
                                object_hook=keys_to_int)

  def puzzle(self, *args, **kwargs) -> Puzzle:
    return Puzzle(*args, **kwargs)

  def show_times(self, recompute=False, repeat=1):
    if recompute and repeat > 1:
      print(f'(Computing min times over {repeat} calls.)')
    total = 0
    for day, puzzle in sorted(self.puzzles.items()):
      s = f'day_{day:<2}'
      for part in (1, 2):
        s += f'   part_{part}:'
        puzzle_part = puzzle.parts.get(part)
        if not puzzle_part:
          s += '  n/a '
        else:
          if recompute:
            puzzle_part.compute(puzzle.input, silent=True, repeat=repeat)
          s += f'{puzzle_part.elapsed_time:#6.3f}'
          total += puzzle_part.elapsed_time
      print(s)
    print(f'Total time:{total:#7.3f}')


advent = Advent(also_run_slow_tests=False)

## Day 1: Find summing numbers

- Part 1: Given a list of numbers, find two entries that sum to 2020 and multiply those entries together.

- Part 2: Find three entries that sum to 2020 and multiply those entries together.

In [5]:
puzzle = advent.puzzle(1)

For day 1, puzzle.input has 200 lines:
1686
1983
1801
1890
1910
1722
1571
1952
 ...
1261
1154
1769
1529
And the stored answers are: {1: 651651, 2: 214486272}


In [6]:
# A test input provided in the puzzle description:
s1 = """
1721
979
366
299
675
1456
"""

In [7]:
def process1(s, total=2020):
  entries = set(map(int, s.split()))
  for a in entries:
    b = total - a
    if b in entries:
      return a * b
  return None


check_eq(process1(s1), 514579)  # reference answer provided in the description
puzzle.verify(1, process1)

(Time:  0.000)


In [8]:
def process2(s, total=2020):
  entries = set(map(int, s.split()))
  for a in entries:
    for b in entries:
      c = total - a - b
      if c in entries:
        return a * b * c
  return None


check_eq(process2(s1), 241861950)
puzzle.verify(2, process2)

(Time:  0.002)


## Day 2: Valid passwords

Given input lines each containing a password rule (`i-j c`) and a password, count the number of "valid" passwords such that:

- Part 1: The number of instances of character `c` in the password is in the range `[i, j]`.

- Part 2: The password characters at positions `i` and `j` contain exactly one instance of character `c`.

In [9]:
puzzle = advent.puzzle(2)

For day 2, puzzle.input has 1000 lines:
4-7 z: zzzfzlzzz
3-4 l: blllk
8-11 j: jjjjjjjgjjjj
6-7 l: lllhlbl
10-13 x: xmhfsbjxptxxv
2-3 s: ssvxszrvj
4-6 r: rtrrrqv
5-6 c: cbccxc
 ...
9-10 w: wwwwwwbwwhww
5-6 g: kgggjg
4-8 s: ssssssscs
12-20 t: ttttttltttvttttttttt
And the stored answers are: {1: 396, 2: 428}


In [10]:
s1 = """
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
"""

In [11]:
def process1(s):
  lines = s.strip('\n').splitlines()
  num_valid = 0
  for line in lines:
    pattern = r'(\d+)-(\d+) (\w): (\w+)'
    vmin, vmax, ch, password = re.fullmatch(pattern, line).groups()
    vmin, vmax = int(vmin), int(vmax)
    count = sum([c == ch for c in password])
    num_valid += vmin <= count <= vmax
  return num_valid


check_eq(process1(s1), 2)
puzzle.verify(1, process1)

(Time:  0.005)


In [12]:
def process2(s):
  lines = s.strip('\n').splitlines()
  num_valid = 0
  for line in lines:
    pattern = r'(\d+)-(\d+) (\w): (\w+)'
    vmin, vmax, ch, password = re.fullmatch(pattern, line).groups()
    vmin, vmax = int(vmin), int(vmax)
    valid = (password[vmin - 1] == ch) ^ (password[vmax - 1] == ch)
    num_valid += valid
  return num_valid


check_eq(process2(s1), 1)
puzzle.verify(2, process2)

(Time:  0.003)


## Day 3: Count intersections in grid

Given a 2D grid containing sparse "trees", determine the number of trees encountered when moving from the top-left to the bottom row with a slope `(dy, dx)` and using wraparound addressing on the horizontal axis.

- Part 1: Count the trees for the slope `(dy, dx) = (1, 3)` 

- Part 2: Compute the product of the tree counts for slopes `(1, 1)`, `(1, 3)`, `(1, 5)`, `(1, 7)`, and `(2, 1)`.

In [13]:
puzzle = advent.puzzle(3)

For day 3, puzzle.input has 323 lines:
........#.............#........
...#....#...#....#.............
.#..#...#............#.....#..#
..#......#..##............###..
..........#......#..#..#.......
.#..#.......#.........#.#......
.........#..#....##..#.##....#.
..#....##...#..................
 ...
..................#...#....#.##
......#.#....#.................
......#.#.....#.....#..##......
#..##...........#..#.....#.##..
And the stored answers are: {1: 148, 2: 727923200}


In [14]:
s1 = """
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
"""


def process(s, dyxs=((1, 3),)):
  grid = np.array(list(map(list, s.strip('\n').splitlines())))

  def get_count(dyx):
    yx = (0, 0)
    count = 0
    while yx[0] < grid.shape[0]:
      count += grid[yx] == '#'
      yx = yx[0] + dyx[0], (yx[1] + dyx[1]) % grid.shape[1]
    return count
  
  return np.prod([get_count(dyx) for dyx in dyxs])


check_eq(process(s1), 7)
puzzle.verify(1, process)

(Time:  0.002)


In [15]:
process2 = functools.partial(
    process, dyxs=((1, 1), (1, 3), (1, 5), (1, 7), (2, 1)))

check_eq(process2(s1), 336)
puzzle.verify(2, process2)

(Time:  0.005)


## Day 4: Valid text fields

Given records containing various fields, count the number of valid records.

- Part 1: Each record is valid if it contains at least some required fields.

- Part 2: The fields also satisfy a set of requirements.

In [16]:
puzzle = advent.puzzle(4)

For day 4, puzzle.input has 1102 lines:
hcl:5d90f0 cid:270 ecl:#66dc9c hgt:62cm byr:1945 pid:63201172 eyr:2026

ecl:amb byr:1943 iyr:2014 eyr:2028
pid:333051831

byr:1971
eyr:2021 iyr:2015 pid:158388040 hcl:#18171d ecl:brn hgt:179cm

 ...
hcl:#7d3b0c pid:431742871
ecl:hzl hgt:169cm cid:340
eyr:2023
iyr:2017 byr:1994
And the stored answers are: {1: 216, 2: 150}


In [17]:
#@title
print('Let s1, s2, s3 be test inputs.')

s1 = """
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
"""

# all invalid
s2 = """
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
"""

# all valid
s3 = """
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
"""

Let s1, s2, s3 be test inputs.


In [18]:
def process1(s):
  required_fields = set('byr iyr eyr hgt hcl ecl pid'.split())
  passports = s.strip('\n').split('\n\n')
  num_valid = 0
  for passport in passports:
    passport = passport.replace('\n', ' ')
    valid = all(field + ':' in passport for field in required_fields)
    num_valid += valid
  return num_valid


check_eq(process1(s1), 2)
puzzle.verify(1, process1)

(Time:  0.001)


In [19]:
# Field requirements:
# byr (Birth Year) - four digits; at least 1920 and at most 2002.
# iyr (Issue Year) - four digits; at least 2010 and at most 2020.
# eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
# hgt (Height) - a number followed by either cm or in:
# If cm, the number must be at least 150 and at most 193.
# If in, the number must be at least 59 and at most 76.
# hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
# ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
# pid (Passport ID) - a nine-digit number, including leading zeroes.
# cid (Country ID) - ignored, missing or not.

In [20]:
def process2(s):
  passports = s.strip('\n').split('\n\n')

  def valid_fields(fields):
    try:
      value, unit = re.fullmatch(r'(\d+)(cm|in)', fields['hgt']).groups()
      year = lambda field: int(re.fullmatch(r'(\d{4})', fields[field]).group())
      return bool(
          1920 <= year('byr') <= 2002 and
          2010 <= year('iyr') <= 2020 and
          2020 <= year('eyr') <= 2030 and
          (unit != 'cm' or 150 <= int(value) <= 193) and
          (unit != 'in' or 59 <= int(value) <= 76) and
          re.fullmatch(r'#[0-9a-f]{6}', fields['hcl']) and
          fields['ecl'] in 'amb blu brn gry grn hzl oth'.split() and
          re.fullmatch(r'[0-9]{9}', fields['pid']))
    except (AttributeError, KeyError):
      return False

  num_valid = 0
  for passport in passports:
    fields = dict(re.fullmatch(r'(\w\w\w):(\S+)', s_field).groups()
                  for s_field in passport.split())
    valid = valid_fields(fields)
    num_valid += valid
  return num_valid


check_eq(process2(s2), 0)  # all records are invalid
check_eq(process2(s3), 4)  # all records are valid
puzzle.verify(2, process2)

(Time:  0.005)


## Day 5: Free seat in grid

Given input lines, each specifiying a seat identified using a binary encoding with symbols **B**ack, **F**ront, **L**eft, **R**ight.

- Part 1: Determine the maximum seat id.

- Part 2: Determine the missing seat id in the middle of the bunch.

In [21]:
puzzle = advent.puzzle(5)

For day 5, puzzle.input has 897 lines:
BFBBFFBRRL
BFBFFFFRLL
FBFBFFBRRR
FFBBBFBRLL
BBBFBFBRLR
BFBBFBFLRR
FBBBFFFRRL
BBFFBFFLLL
 ...
BBFBFBFRRL
BFFBBBBLLL
FBFFBFBRRR
FBFFFFBRLR
And the stored answers are: {1: 965, 2: 524}


In [22]:
def get_seat_id(line):
  return int(line.translate(str.maketrans('FBLR', '0101')), base=2)

check_eq(get_seat_id('FBFBBFFRLR'), 357)
check_eq(get_seat_id('BFFFBBFRRR'), 567)
check_eq(get_seat_id('FFFBBBFRRR'), 119)
check_eq(get_seat_id('BBFFBBFRLL'), 820)


def process1(s):
  return max(get_seat_id(line) for line in s.split())

puzzle.verify(1, process1)

(Time:  0.001)


Part 2

In [23]:
# For helpful visualization:
def visualize_transposed_seat_grid(s):
  grid = np.full((128, 8), 0)
  yx = np.array([divmod(get_seat_id(line), 8) for line in s.split()])
  grid[tuple(yx.T)] = 1
  print('\n'.join(''.join('.#'[e] for e in row) for row in grid.T))

visualize_transposed_seat_grid(puzzle.input)

.........################################################################################################################.......
.........################################################################################################################.......
.........################################################################################################################.......
.........################################################################################################################.......
........#########################################################.#######################################################.......
........#################################################################################################################.......
........################################################################################################################........
........#########################################################################################

In [24]:
# Code using string indexing:

def find_all(string, substring, overlapping=False):
  i = string.find(substring)
  while i != -1:
    yield i
    i = string.find(substring, i + (1 if overlapping else len(substring)))

def process2(s):
  seat_ids = [get_seat_id(line) for line in s.split()]
  occupied = ''.join('01'[seat_id in seat_ids] for seat_id in range(128 * 8))
  i, = list(find_all(occupied, '101'))
  seat_id = i + 1
  return seat_id

puzzle.verify(2, process2)

(Time:  0.007)


In [25]:
# Code using regular expression search:
def process2(s):
  seat_ids = [get_seat_id(line) for line in s.split()]
  occupied = ''.join('01'[seat_id in seat_ids] for seat_id in range(128 * 8))
  seat_id, = [match.start() for match in re.finditer('(?<=1)0(?=1)', occupied)]
  return seat_id

puzzle.verify(2, process2)

(Time:  0.010)


In [26]:
# Code using numpy successive differences of sorted indices (fast):
def process2(s):
  seat_ids = np.sort([get_seat_id(line) for line in s.split()])
  diff = np.diff(seat_ids)
  i, = np.nonzero(diff == 2)[0]
  seat_id = seat_ids[i] + 1
  return seat_id

puzzle.verify(2, process2)

(Time:  0.002)


In [27]:
# Code using numpy subsequence search (also fast):

def matching_subsequences(array, sequence):
  array, sequence = np.asarray(array), np.asarray(sequence)
  return (array[np.arange(len(array) - len(sequence) + 1)[:, None] +
          np.arange(len(sequence))] == sequence).all(axis=1)

def process2(s):
  seat_ids = [get_seat_id(line) for line in s.split()]
  occupied = np.full(128 * 8, False)
  occupied[seat_ids] = True
  seat_id, = np.nonzero(matching_subsequences(occupied, (1, 0, 1)))[0] + 1
  return seat_id

puzzle.verify(2, process2)

(Time:  0.001)


## Day 6: Counts across groups

Given a set of records of words:

- Part 1: Compute the number of unique letters in each record, and sum these.

- Part 2: Compute the number of letters appearing in all words of each record, and sum these.

In [28]:
puzzle = advent.puzzle(6)

For day 6, puzzle.input has 2172 lines:
su
egu

h
h
h
h

 ...

lfhtzrxcj
fbtzlhrcj
lzdaftrjphco
And the stored answers are: {1: 6590, 2: 3288}


In [29]:
#@title
print('Let s1 be a test input.')

s1 = """
abc

a
b
c

ab
ac

a
a
a
a

b
"""

Let s1 be a test input.


Part 1

In [30]:
# Long code:
def process1(s):
  total = 0
  for group in s.strip('\n').split('\n\n'):
    union = set()
    for line in group.splitlines():
      union |= set(line)
    total += len(union)
  return total

check_eq(process1(s1), 11)
puzzle.verify(1, process1)

(Time:  0.003)


In [31]:
# Code using reduction:
def process1(s):
  return sum(
      len(functools.reduce(operator.or_, map(set, group.splitlines())))
      for group in s.strip('\n').split('\n\n'))

check_eq(process1(s1), 11)
puzzle.verify(1, process1)

(Time:  0.005)


In [32]:
# Compact code which is also fastest:
def process1(s):
  return sum(
      len(set().union(group.replace('\n', '')))
      for group in s.strip('\n').split('\n\n'))

check_eq(process1(s1), 11)
puzzle.verify(1, process1)

(Time:  0.001)


Part 2

In [33]:
# Code using reduction:
def process2(s):
  return sum(
      len(functools.reduce(operator.and_, map(set, group.splitlines())))
      for group in s.strip('\n').split('\n\n'))

check_eq(process2(s1), 6)
puzzle.verify(2, process2)

(Time:  0.004)


In [34]:
# Compact code which is faster:
def process2(s):
  return sum(
      len(set.intersection(*map(set, group.splitlines())))
      for group in s.strip('\n').split('\n\n'))

check_eq(process2(s1), 6)
puzzle.verify(2, process2)

(Time:  0.003)


In [35]:
# Long code which is fastest:
def process2(s):
  total = 0
  for group in s.strip('\n').split('\n\n'):
    intersect = None
    for line in group.splitlines():
      line_set = set(line)
      if intersect is None:
        intersect = line_set
      else:
        intersect &= line_set
        if not intersect:
          break
    total += len(intersect)
  return total

check_eq(process2(s1), 6)
puzzle.verify(2, process2)

(Time:  0.002)


## Day 7: Nested bags

Given a description of bag colors, where each colored bag contains a collection of other colored bags:

- Part 1: Determine the number of bag colors that can eventually contain a specified bag color.

- Part 2: Determine the total number of bags required inside a specified bag color.

In [36]:
puzzle = advent.puzzle(7)

For day 7, puzzle.input has 594 lines:
dim red bags contain 2 bright gold bags, 5 striped fuchsia bags.
dotted purple bags contain 5 bright olive bags, 3 faded maroon bags.
plaid chartreuse bags contain 1 vibrant olive bag, 5 bright black bags, 1 clear tomato bag.
wavy orange bags contain 4 dark lavender bags, 4 posh white bags.
light lavender bags contain 4 drab olive bags, 5 dark magenta bags.
pale black bags contain 5 dotted beige bags, 5 clear chartreuse bags, 4 pale lime bags.
wavy crimson bags contain 5 dull tomato bags.
muted aqua bags contain 1 dotted aqua bag, 5 posh plum bags, 1 bright blue bag.
 ...
vibrant lime bags contain 5 light green bags.
dark maroon bags contain 5 drab bronze bags, 4 bright red bags, 1 posh cyan bag.
wavy red bags contain 2 mirrored beige bags, 5 muted crimson bags, 5 vibrant orange bags, 3 posh turquoise bags.
drab brown bags contain 1 faded tan bag, 3 vibrant maroon bags.
And the stored answers are: {1: 115, 2: 1250}


In [37]:
#@title
print('Let s1, s2 be test inputs.')

s1 = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""

s2 = """
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
"""

Let s1, s2 be test inputs.


In [38]:
def get_bag_contents(s):
  contents = {}
  for line in s.strip('\n').splitlines():
    outer, inners = line.split(' bags contain ')
    contents[outer] = {}
    for match in re.finditer(r'(\d+) (.*?) bags?[,.]', inners):
      n, inner = match.groups()
      contents[outer][inner] = int(n)
  return contents

In [39]:
# Compact code
def process1(s, query='shiny gold'):
  """Returns number of bag colors that can eventually contain >=1 query bag."""
  contents = get_bag_contents(s)

  def valid(bag):
    return any(inner == query or valid(inner) for inner in contents[bag])

  return sum(valid(bag) for bag in contents)

check_eq(process1(s1), 4)
puzzle.verify(1, process1)

(Time:  0.200)


In [40]:
# Faster code
def process1(s, query='shiny gold'):
  contents = get_bag_contents(s)  # computational bottleneck
  parents = collections.defaultdict(list)
  for bag in contents:
    for child in contents[bag]:
      parents[child].append(bag)

  valid = set()
  to_visit = collections.deque([query])
  while to_visit:
    bag = to_visit.popleft()
    for parent in parents[bag]:
      if parent not in valid:
        valid.add(parent)
        to_visit.append(parent)

  return len(valid)

check_eq(process1(s1), 4)
puzzle.verify(1, process1)

(Time:  0.003)


Part 2

In [41]:
def process2(s, query='shiny gold'):
  """Returns total number of individual bags required inside a query bag."""
  contents = get_bag_contents(s)  # computational bottleneck

  def count_inside(bag):
    return sum(n * (count_inside(b) + 1) for b, n in contents[bag].items())

  return count_inside(query)

check_eq(process2(s1), 32)
check_eq(process2(s2), 126)
puzzle.verify(2, process2)

(Time:  0.003)


## Day 8: Program with acc, nop, jmp

Given a program with instructions `nop`, `acc`, and `jmp`:

- Part 1: Determine the sum of the `acc` operands when running the program until the program counter becomes invalid.

- Part 2: Modify the program by swapping one of its opcodes between `nop` and `jmp` such that it terminates (with the program counter at the end), and report the sum of the `acc` operands.

In [42]:
puzzle = advent.puzzle(8)

For day 8, puzzle.input has 638 lines:
acc +8
acc +37
jmp +328
jmp +574
nop +321
acc +17
jmp +450
acc +46
 ...
acc +6
acc +21
acc +16
jmp +1
And the stored answers are: {1: 1521, 2: 1016}


In [43]:
s1 = """
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
"""

In [44]:
def run_program(ops):
  pc = 0
  acc = 0
  visited = set()
  while 0 <= pc < len(ops) and pc not in visited:
    visited.add(pc)
    operation, operand = ops[pc]
    if operation == 'nop':
      pc += 1
    elif operation == 'acc':
      acc += operand
      pc += 1
    elif operation == 'jmp':
      pc += operand
    else:
      raise AssertionError
  return pc, acc


def process1(s):
  ops = [(line[:3], int(line[4:])) for line in s.strip('\n').splitlines()]
  pc, acc = run_program(ops)
  assert 0 <= pc < len(ops)
  return acc

check_eq(process1(s1), 5)
puzzle.verify(1, process1)

(Time:  0.000)


In [45]:
def process2(s):
  ops = [(line[:3], int(line[4:])) for line in s.strip('\n').splitlines()]
  for i in range(len(ops)):
    if ops[i][0] in ('nop', 'jmp'):
      original_op = ops[i]
      ops[i] = (('jmp' if ops[i][0] == 'nop' else 'nop'), ops[i][1])
      pc, acc = run_program(ops)
      ops[i] = original_op
      if pc == len(ops):
        return acc
  return None

check_eq(process2(s1), 8)
puzzle.verify(2, process2)

(Time:  0.010)


## Day 9: Sums in subsequences

Given a list of numbers:

- Part 1: Find the first number that is not a sum of a pair of numbers in the prior `n` numbers.

- Part 2: Find a subsequence of at least 2 numbers that sums to the value obtained in part 1, compute the min and max of the subsequence, and report their product.

In [46]:
puzzle = advent.puzzle(9)

For day 9, puzzle.input has 1000 lines:
19
30
13
31
42
41
44
34
 ...
30187342902183
39869127705106
31005328029673
49124948203304
And the stored answers are: {1: 393911906, 2: 59341885}


In [47]:
#@title
print('Let s1 be a test input.')

s1 = """
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
"""

Let s1 be a test input.


In [48]:
def process(s, last_n=25, part2=False):
  l = list(map(int, s.split()))

  def has_pair(l, total):
    # return any(a + b == total for a, b in itertools.combinations(l, 2))
    return any(total - l[i] in l[i + 1:] for i in range(len(l)))

  def first_number_not_sum_of_pair_in_last_n(n):
    return next(l[i] for i in range(n, len(l)) 
                if not has_pair(l[i - n:i], l[i]))

  invalid_number = first_number_not_sum_of_pair_in_last_n(last_n)
  if not part2:
    return invalid_number

  # Find a contiguous set of at least two numbers in your list which sum to the
  # invalid number.  Approach: compute cumulative sum accum[i], then for each i
  # do bisection search to try to locate l[i] + invalid_number.

  def find_subsequence_summing_to(total):
    accum = list(itertools.accumulate(l, operator.add))  # accum[i] = sum(l[:i])
    for i in range(len(accum)):
      end_value = accum[i] + total
      j = bisect.bisect_left(accum, end_value)
      if j != len(accum) and accum[j] == end_value:
        return l[i + 1:j + 1]

  sequence = find_subsequence_summing_to(invalid_number)
  assert sum(sequence) == invalid_number
  return min(sequence) + max(sequence)


check_eq(process(s1, last_n=5), 127)
puzzle.verify(1, process)

process2 = functools.partial(process, part2=True)
check_eq(process2(s1, last_n=5), 62)
puzzle.verify(2, process2)

(Time:  0.002)
(Time:  0.002)


## Day 10: Combinations of sequences

Given a set of numbers,

- Part 1: Augment the numbers with the values `0` and `max + 3`, tally the successive differences of the sorted numbers, and report the product of the 1-differences and the 3-differences.

- Part 2: Count the number of ways to monotonically traverse the numbers such that successive differences are at most 3.

In [49]:
puzzle = advent.puzzle(10)

For day 10, puzzle.input has 97 lines:
8
131
91
35
47
116
105
121
 ...
95
3
52
49
And the stored answers are: {1: 2112, 2: 3022415986688}


In [50]:
#@title
print('Let s1, s2 be test inputs.')

s1 = """
16
10
15
5
1
11
7
19
6
12
4
"""

s2 = """
28
33
18
42
31
14
46
20
48
47
24
23
49
45
19
38
39
11
1
32
25
35
8
17
7
9
4
2
34
10
3
"""

Let s1, s2 be test inputs.


In [51]:
def process1(s):
  l = sorted(map(int, s.split()))
  counter = collections.Counter(np.diff([0] + l + [l[-1] + 3]))
  return counter[1] * counter[3]

check_eq(process1(s1), 7 * 5)
check_eq(process1(s2), 22 * 10)
puzzle.verify(1, process1)  # 66 * 32

(Time:  0.000)


Part 2

In [52]:
# Observation: when there is a separation of 3 units, the problem decouples
# into a product of the solutions of subproblems.

# Because the voltages differences are just 1 or 3, we can simply count the
# length of the sequences of value 1, and derive a closed-form number for each
# sequence.

def process2(s):
  diff = np.diff([0] + sorted(map(int, s.split())))
  lengths_of_ones = map(len, re.findall('1+', ''.join(map(str, diff))))

  def f(n):
    """Number of combinations from a sequence of n consecutive one-diffs."""
    return 0 if n < 0 else 1 if n < 2 else f(n - 1) + f(n - 2) + f(n - 3)

  return np.prod(list(map(f, lengths_of_ones)))

check_eq(process2(s1), 8)
check_eq(process2(s2), 19208)
puzzle.verify(2, process2)

(Time:  0.000)


In [53]:
# More general solution based on dynamic programming with scatter.
def process2(s):
  l = [0] + sorted(map(int, s.split()))
  npaths = [1] + [0] * (len(l) - 1)
  for i in range(len(l)):
    for j in range(i + 1, min(i + 4, len(l))):
      if l[j] - l[i] <= 3:
        npaths[j] += npaths[i]
  return npaths[-1]

check_eq(process2(s1), 8)
check_eq(process2(s2), 19208)
puzzle.verify(2, process2)

(Time:  0.000)


In [54]:
# Simplest solution based on dynamic programming with gather.
def process2(s):
  l = [0] + sorted(map(int, s.split()))
  npaths = [1]
  for i in range(1, len(l)):
    npaths.append(sum(npaths[j] for j in range(-4, 0)
                      if i + j >= 0 and l[i] - l[i + j] < 4))
  return npaths[-1]

check_eq(process2(s1), 8)
check_eq(process2(s2), 19208)
puzzle.verify(2, process2)

(Time:  0.000)


## Day 11: Seats cellular automaton

Given a *sparse* 2D grid of nodes, run successive generations of a cellular automaton until convergence.

- Part 1: Each node's neighbors are the immediately adjacent nodes (up to 8, with Manhattan distance 1).
  
  For each generation:
  - a free node becomes occupied if it has zero occupied neighbors, and
  - an occupied node becomes free if it has 4 or more occupied neighbors.

- Part 2: The evolution rule is modified in two ways:
  - For each of the 8 neighbor directions, the neighbor is defined as the first node along that direction.
  - An occupied node becomes free if it has 5 or more occupied neighbors.

In [55]:
puzzle = advent.puzzle(11)

For day 11, puzzle.input has 99 lines:
LLLLLLLLLL.LLLLLL.LLLLLLLLLLLL.LL.LLLL.LLLLL.LLLLLLL.LLLLLL.LLLLLL.LLLLLL.LLLLLLLLLLLLLLLLLL
LLLLLLLLLLLLLLLLLLLLLLLLLLLLLL.LLLLLLL.LLLLL.LLLLLLL.LLLLLLLLLLLLL.LLLL.L.LLLLLLLLLL.LLLLLLL
LLLLLLLLLL.LLLLLL.LLLLLL.LLLLL.LLLLLLLLLLLLLLLLLLLLL.LLLLLL..LLLLL.LLLLLL.LLLLLL.LLLLLLLLLLL
LLLLLLLLLL.LLLLLLLLLLLLL.LL.LL.LLLLL.L.LLLLL.LLLLLLL.LLLLLL.LLLLLLLLLLLLL.LLLLLLLLLLLLLLLLLL
LLLLLLLLLLLLLLLLL.LLLLLL.LLLLLLLLLLLLLLLLLLL.LLLLLLL.LLL.LL.LLLLLL.LLLLLL.LLLLLLLLLLLLLLLLLL
LLLLLLLLLLLLLLLLL.LLLLLL.LLLLL.LLLLLLL.LLL.L.LLLLLLLLLLLLLLLLLLLLLLLLLLLL.LLLLLL.LLLLLLLL.LL
LLLLLLLLLLLLLLLLL.LLLLLLLLLL.LLLLLLLLL.LLLLL.LLLLLLL.LLLL.L.LLLLLLLLLLLLL.LLLLLLLLLLLLLLLLLL
.L.LL...LLLL.......L....L.LLLLLL.......LL....LL...L..L.LLL...LLL..L.L.L.L..L...............L
 ...
LLLLLLLLLL.LLLLLL.LLLLLL.LLLLL.LLLLL.L.LLLLLLLLLLLLL.LL.LLLLL.LLLL.LLLLLL.LLLLLLLLLL.LLLLLLL
LLLLLLLLLLLLLLLLL.LLLLLL.LLLLL.LLLLLLL.LLLLL..LLLLLL.LLLLLL.LLLLLL.LLLLLL.LLLLLL.LLLLLLLLLLL
LLLLLLLLLL.LLL.LLLLLLLLL.L

In [56]:
s1 = """
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""

In [57]:
# Relatively fast non-numba solution, which maintains neighbor counts.
# It becomes faster with numba enabled, but not as fast as the next solution.

def process(s, part2=False):
  lines = s.strip('\n').split('\n')
  # -1 is EMPTY, 0..8 is FREE+neighbor_count, 10..18 is OCCUPIED+neighbor_count
  from_ch = lambda ch: {'.': -1, 'L': 0, '#': 10}[ch]
  grid = np.array([[from_ch(ch) for ch in line] for line in lines])
  # Surprisingly, the default array dtype (here, np.int64) is the fastest.
  shape = grid.shape
  NEIGHBORS = tuple(set(itertools.product((-1, 0, 1), repeat=2)) - {(0, 0)})

  def evolve(grid):

    def impact_neighbor_counts(y0, x0, delta):
      for dy, dx in NEIGHBORS:
        y, x = y0 + dy, x0 + dx
        while 0 <= y < shape[0] and 0 <= x < shape[1]:
          if not part2 or grid[y, x] >= 0:
            if grid[y, x] >= 0:
              grid[y, x] += delta
            break
          y, x = y + dy, x + dx

    # print('num_occupied', np.sum(grid >= 10))
    prev = grid.copy()
    for y in range(shape[0]):
      for x in range(shape[1]):
        if prev[y, x] == 0:  # FREE with zero occupied neighbors
          grid[y, x] += 10
          impact_neighbor_counts(y, x, +1)
        elif prev[y, x] >= (15 if part2 else 14):  # OCCUPIED, >= 5 or 4 neighb
          grid[y, x] -= 10
          impact_neighbor_counts(y, x, -1)
    return np.any(grid != prev)

  while evolve(grid):
    pass
  return np.sum(grid >= 10)


check_eq(process(s1), 37)

In [58]:
# More naive solution but which is faster when using numba.

if using_numba:

  def process(s, part2=False):
    lines = s.strip('\n').split('\n')
    from_ch = lambda ch: {'.': 0, 'L': 1, '#': 2}[ch]
    grid = np.array([[from_ch(ch) for ch in line] for line in lines])
    neighbors = tuple(set(itertools.product((-1, 0, 1), repeat=2)) - {(0, 0)})

    @numba_njit(cache=True, parallel=True)
    def evolve(grid, neighbors, part2):
      EMPTY, FREE, OCCUPIED = 0, 1, 2
      shape = grid.shape

      def count_occupied_neighbors(grid, y0, x0, only_adjacent):
        count = 0
        for dy, dx in neighbors:
          y, x = y0 + dy, x0 + dx
          while 0 <= y < shape[0] and 0 <= x < shape[1]:
            if grid[y, x] != EMPTY or only_adjacent:
              count += grid[y, x] == OCCUPIED
              break
            y, x = y + dy, x + dx
        return count

      while True:
        # print('num_occupied', np.sum(grid == 2))
        prev = grid.copy()
        for y in range(shape[0]):
          for x in range(shape[1]):
            num_occupied = count_occupied_neighbors(prev, y, x, not part2)
            if prev[y, x] == FREE and num_occupied == 0:
              grid[y, x] = OCCUPIED
            elif prev[y, x] == OCCUPIED and num_occupied >= (5 if part2 else 4):
              grid[y, x] = FREE
        if np.all(grid == prev):
          break

      return np.sum(grid == OCCUPIED)

    return evolve(grid, neighbors, part2)

  check_eq(process(s1), 37)

In [59]:
# ~2 s for non-numba solution; ~0.05 s for numba solution
puzzle.verify(1, process)

(Time:  0.042)


In [60]:
process2 = functools.partial(process, part2=True)
check_eq(process2(s1), 26)

# ~5 s for non-numba solution; ~0.05 s for numba solution
puzzle.verify(2, process2)

(Time:  0.047)


## Day 12: Ship and waypoint navigation

Given a list of instructions (a translation direction `N|S|E|W` and magnitude, a rotation `L|R` and angle (positive multiple of 90 degrees), or a forward directive `F` and value), update a ship's position and orientation in a 2D domain.

- Part 1: The translation, rotation, and forward instructions directly modify the ship's state.  After applying all instructions, report the Manhattan distance of the ship's position from its start.

- Part 2: The translation and rotation instructions modify a vector waypoint (initially 10 units east and 1 unit north of ship) relative to the ship position.  The forward instruction translates the ship position by the vector waypoint times the forward value.  Again report the final Manhattan distance of the ship from its start.

In [61]:
puzzle = advent.puzzle(12)

For day 12, puzzle.input has 773 lines:
W5
F63
S1
L90
F89
W4
F45
W4
 ...
L90
F94
L90
F13
And the stored answers are: {1: 1221, 2: 59435}


In [62]:
s1 = """
F10
N3
F7
R90
F11
"""

In [63]:
def process1(s):
  y, x = 0, 0
  dy, dx = 0, 1  # east

  for instruction in s.split():
    action, value = instruction[:1], int(instruction[1:])
    if action == 'N':
      y -= value
    elif action == 'S':
      y += value
    elif action == 'E':
      x += value
    elif action == 'W':
      x -= value
    elif action == 'L':
      while value:
        dy, dx = (-dx, +dy)
        value -= 90
    elif action == 'R':
      while value:
        dy, dx = (+dx, -dy)
        value -= 90
    elif action == 'F':
      y, x = y + dy * value, x + dx * value
  return abs(y) + abs(x)


check_eq(process1(s1), 25)
puzzle.verify(1, process1)

(Time:  0.000)


In [64]:
def process2(s):
  ship_y, ship_x = 0, 0
  # waypoint is relative to the ship
  waypoint_y, waypoint_x = -1, 10  # 10 units east and 1 unit north of ship

  for instruction in s.split():
    action, value = instruction[:1], int(instruction[1:])
    if action == 'N':
      waypoint_y -= value
    elif action == 'S':
      waypoint_y += value
    elif action == 'E':
      waypoint_x += value
    elif action == 'W':
      waypoint_x -= value
    elif action == 'L':
      while value:
        waypoint_y, waypoint_x = -waypoint_x, +waypoint_y
        value -= 90
    elif action == 'R':
      while value:
        waypoint_y, waypoint_x = +waypoint_x, -waypoint_y
        value -= 90
    elif action == 'F':
      ship_y += waypoint_y * value
      ship_x += waypoint_x * value
  return abs(ship_y) + abs(ship_x)


check_eq(process2(s1), 286)
puzzle.verify(2, process2)

(Time:  0.000)


## Day 13: Buses modulo schedule

Given a sequence of buses with different departure periods, all starting at time 0, with some buses out-of-service, find some earliest time satisfying a condition.

- Part 1: Given also an earliest time, find the first bus that departs on or after that time.  Report the wait time multiplied by the bus period.

- Part 2: Find the earliest time such that the first bus in the sequence departs at that time and each subsequent listed bus departs at the subsequent time if the bus is in service.

In [65]:
puzzle = advent.puzzle(13)

For day 13, puzzle.input has 2 lines:
1000299
41,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,37,x,x,x, ... ,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,19
And the stored answers are: {1: 156, 2: 404517869995362}


In [66]:
s1 = """
939
7,13,x,x,59,x,31,19
"""

In [67]:
def ceil_of_ratio(a: int, b: int) -> int:
  """Returns the ceiling of the ratio a / b, where b must be positive."""
  assert b > 0
  return (a + b - 1) // b


def process1(s):
  lines = s.strip('\n').splitlines()
  earliest_time = int(lines[0])
  buses = [int(e) for e in lines[1].split(',') if e != 'x']
  next_times = [ceil_of_ratio(earliest_time, bus) * bus for bus in buses]
  min_time, min_bus = min(zip(next_times, buses))
  return (min_time - earliest_time) * min_bus


check_eq(process1(s1), 295)
puzzle.verify(1, process1)

(Time:  0.000)


Part 2

In [68]:
def extended_gcd(a: int, b: int) -> Tuple[int, int, int]:
  """Finds the greatest common divisor using the extended Euclidean algorithm.

  Returns:
    (gcd(a, b), x, y) with the property that a * x + b * y = gcd(a, b).

  >>> extended_gcd(29, 71)
  (1, -22, 9)
  >>> (29 * -22) % 71
  1
  """
  prev_x, x = 1, 0
  prev_y, y = 0, 1
  while b:
    q = a // b
    x, prev_x = prev_x - q * x, x
    y, prev_y = prev_y - q * y, y
    a, b = b, a % b
  x, y = prev_x, prev_y
  return a, x, y

In [69]:
# Let each bus be represented by its period b_i and desired remainder r_i.

# Given [(b_i, r_i)], where b_i are coprime, we seek x such that
#  x % b_i = r_i  for all i,  where r_i = -i % b_i
# We can work by reduction, merging pairs of buses.

# Given a pair of buses (b1, r1) and (b2, r2),
# we want to find a new equivalent bus (b1 * b2, r) such that
#  0 <= r < b1 * b2
#  r % b1 = r1
#  r % b2 = r2
# We can apply the Chinese remainder theorem using the extended GCD algorithm.

In [70]:
def process2(s):
  s = s.strip('\n').splitlines()[-1]
  buses = [int(e) for e in s.replace('x', '1').split(',')]
  check_eq(np.lcm.reduce(buses), np.prod(buses))  # verify all coprime
  bus_remainders = [(bus, -i % bus) for i, bus in enumerate(buses) if bus > 1]

  def merge_buses(bus1, bus2):
    (b1, r1), (b2, r2) = bus1, bus2
    # https://en.wikipedia.org/wiki/Chinese_remainder_theorem
    _, x, y = extended_gcd(b1, b2)
    return b1 * b2, (r1 * y * b2 + r2 * x * b1) % (b1 * b2)

  _, r = functools.reduce(merge_buses, bus_remainders)
  return r


check_eq(process2(s1.strip('\n').splitlines()[1]), 1068781)
check_eq(process2('17,x,13,19'), 3417)
check_eq(process2('67,7,59,61'), 754018)
check_eq(process2('67,x,7,59,61'), 779210)
check_eq(process2('67,7,x,59,61'), 1261476)
check_eq(process2('1789,37,47,1889'), 1202161486)
puzzle.verify(2, process2)

(Time:  0.000)


## Day 14: Write masked values to mem

Process a sequence of instructions, each specifying either a bitmask (where each location contains `0`, `1`, or `X`) or the writing of a value to a memory address, and report the sum of the resulting memory contents.

- Part 1: The bitmask is applied to the value written to memory, such that `0` and `1` replace the corresponding bit in the value and `X` leaves it unchanged.

- Part 2: The bitmask is applied to the address used to write the value to memory, such that `1` replaces the corresponding bit in the address and `X` indicates a wildcard.  The value is written to all memory addresses satisfying the wildcard.

In [71]:
puzzle = advent.puzzle(14)

For day 14, puzzle.input has 570 lines:
mask = 0111X10100100X1111X10010X000X1000001
mem[50907] = 468673978
mem[22295] = 3337449
mem[58474] = 56418393
mem[15362] = 243184
mem[65089] = 110688658
mask = 010X010XX110X01X01X10X001001011X110X
mem[21952] = 950257
 ...
mem[13666] = 52961
mem[7377] = 134144120
mem[58224] = 211536
mem[36878] = 173058887
And the stored answers are: {1: 9628746976360, 2: 4574598714592}


In [72]:
s1 = """
mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0
"""

In [73]:
def process1(s):
  mem = {}
  extract_0 = str.maketrans('01X', '100')
  extract_1 = str.maketrans('01X', '010')
  for line in s.strip('\n').splitlines():
    if line.startswith('mask'):
      mask = line.split(' = ')[1]
      assert len(mask) == 36
      mask_force_0 = int(mask.translate(extract_0), 2)
      mask_force_1 = int(mask.translate(extract_1), 2)
    else:
      address, value = map(
          int, re.fullmatch(r'mem\[(\d+)\] = (\d+)', line).groups())
      mem[address] = (value | mask_force_1) & ~mask_force_0
  return sum(mem.values())


check_eq(process1(s1), 165)
puzzle.verify(1, process1)

(Time:  0.002)


Part 2

In [74]:
s2 = """
mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1
"""

In [75]:
def process2(s):
  mem = {}
  extract_1 = str.maketrans('01X', '010')
  extract_x = str.maketrans('01X', '001')
  for line in s.strip('\n').splitlines():
    if line.startswith('mask'):
      mask = line.split(' = ')[1]
      assert len(mask) == 36
      mask_force_1 = int(mask.translate(extract_1), 2)
      mask_not_x = ~int(mask.translate(extract_x), 2)
      offsets = [0]
      for wildcard in (1 << i for i in range(36) if mask[35 - i] == 'X'):
        offsets += [wildcard + e for e in offsets]
    else:
      address, value = map(
          int, re.fullmatch(r'mem\[(\d+)\] = (\d+)', line).groups())
      address = (address | mask_force_1) & mask_not_x
      for offset in offsets:
        mem[address + offset] = value
  return sum(mem.values())


check_eq(process2(s2), 208)
puzzle.verify(2, process2)

(Time:  0.022)


## Day 15: Rounds of spoken numbers

Given an initial sequence of numbers indexed starting at `1`, continue the sequence as follows.

The number at index `i` is either
  * `0` if the numer at index `i - 1` was not previously seen, or
  * `i - 1 - j` if the number at index `i - 1` was previously seen at index `j`.

- Part 1: Report the number at index `2020`.

- Part 2: Report the number at index `30_000_000`.

In [76]:
puzzle = advent.puzzle(15)

For day 15, puzzle.input has 1 lines:
9,12,1,4,17,0,18
And the stored answers are: {1: 610, 2: 1407}


In [77]:
s1 = '0,3,6'

In [78]:
# Code that does not use numba:

def generate_sequence(initial_sequence):
  last_turn = {}
  for turn, number in enumerate(initial_sequence):
    prev_turn = last_turn.get(number, -1)
    last_turn[number] = turn
    yield number

  for turn in itertools.count(len(initial_sequence)):
    number = 0 if prev_turn < 0 else turn - 1 - prev_turn
    prev_turn = last_turn.get(number, -1)
    last_turn[number] = turn
    yield number


def process(s, num_turns=2020):
  initial_sequence = list(map(int, s.split(',')))
  return next(itertools.islice(generate_sequence(
      initial_sequence), num_turns - 1, None))


check_eq(process(s1, num_turns=10), 0)
check_eq(process(s1), 436)
check_eq(process('1,3,2'), 1)
check_eq(process('2,1,3'), 10)
check_eq(process('1,2,3'), 27)
check_eq(process('2,3,1'), 78)
check_eq(process('3,2,1'), 438)
check_eq(process('3,1,2'), 1836)
puzzle.verify(1, process)

(Time:  0.001)


In [79]:
if 0:  # The output looks chaotic, without any obvious pattern.
  pprint.pprint(list(itertools.islice(generate_sequence(s1.split(',')), 1000)),
                width=150, compact=True)

In [80]:
# Code which is faster but only if numba is enabled:
if using_numba:
  def process(s, num_turns=2020):

    @numba_njit(cache=True)
    def func(initial_sequence, num_turns):
      last_turn = np.full(num_turns, -1, dtype=np.int32)
      for turn in range(min(num_turns, len(initial_sequence))):
        number = initial_sequence[turn]
        prev_turn = last_turn[number]
        last_turn[number] = turn

      for turn in range(len(initial_sequence), num_turns):
        number = 0 if prev_turn < 0 else turn - 1 - prev_turn
        prev_turn = last_turn[number]
        last_turn[number] = turn

      return number

    initial_sequence = np.array(list(map(int, s.split(','))), dtype=np.int32)
    return func(initial_sequence, num_turns)

process2 = functools.partial(process, num_turns=30_000_000)

if using_numba:
  check_eq(process2(s1), 175594)
  if 0:
    check_eq(process2('1,3,2'), 2578)
    check_eq(process2('2,1,3'), 3544142)
    check_eq(process2('1,2,3'), 261214)
    check_eq(process2('2,3,1'), 6895259)
    check_eq(process2('3,2,1'), 18)
    check_eq(process2('3,1,2'), 362)

# ~1 s with numba, ~20 s for non-numba version
puzzle.verify(2, process2)

(Time:  0.786)


## Day 16: Match ticket fields and rules

Given (1) a set of fields with names and rules on possible value ranges, (2) my ticket with unlabled fields, and (3) a set of other tickets with unlabeled fields:

- Part 1: Report the number of invalid other tickets, which are those containing a field value that does not satisfy any field rule.

- Part 2: Considering the valid other tickets, determine a labeling of the ticket fields that is compatible with the rules.  Report the product of the 6 field values on my ticket whose names start with `departure`.

In [81]:
puzzle = advent.puzzle(16)

For day 16, puzzle.input has 268 lines:
departure location: 33-679 or 691-971
departure station: 48-646 or 671-966
departure platform: 37-601 or 619-950
departure track: 41-863 or 875-973
departure date: 37-145 or 168-965
departure time: 26-246 or 257-972
arrival location: 30-542 or 556-960
arrival station: 30-75 or 89-954
 ...
845,453,939,349,884,940,0,621,933,188,73,204,440,918,207,671,629,735,487,627
879,694,125,501,580,274,451,885,834,397,146,104,185,173,528,717,590,412,350,936
944,91,450,631,262,501,196,401,199,946,105,236,634,765,66,385,582,437,262,581
924,594,156,702,128,858,133,695,140,486,399,534,115,627,748,560,518,700,883,206
And the stored answers are: {1: 29878, 2: 855438643439}


In [82]:
#@title
print('Let s1, s2 be test inputs.')

s1 = """
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
"""

s2 = """
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
"""

Let s1, s2 be test inputs.


In [83]:
def read_rules_and_tickets(s):
  s_rules, s_my_ticket, s_nearby = s.strip('\n').split('\n\n')

  def read_rules():
    for line in s_rules.splitlines():
      name, ranges = line.split(': ')
      ranges = ranges.split(' or ')
      yield (name, [tuple(map(int, range.split('-'))) for range in ranges])

  line, = s_my_ticket.splitlines()[1:]
  my_ticket = list(map(int, line.split(',')))

  def read_tickets():
    for line in s_nearby.splitlines()[1:]:
      fields = list(map(int, line.split(',')))
      yield fields

  return read_rules, my_ticket, read_tickets


def value_ok_for_some_rule(value, rules):
  return any(range[0] <= value <= range[1] 
             for ranges in rules.values() for range in ranges)


def process1(s):
  read_rules, unused_my_ticket, read_tickets = read_rules_and_tickets(s)
  rules = dict(read_rules())
  sum_invalid_values = 0
  for fields in read_tickets():
    for value in fields:
      if not value_ok_for_some_rule(value, rules):
        sum_invalid_values += value
  return sum_invalid_values


check_eq(process1(s1), 71)
puzzle.verify(1, process1)

(Time:  0.005)


Part 2

In [84]:
def process2(s):
  read_rules, my_ticket, read_tickets = read_rules_and_tickets(s)
  rules = dict(read_rules())
  valid_tickets = [
      fields for fields in read_tickets()
      if all(value_ok_for_some_rule(value, rules) for value in fields)
  ]
  num_rules, num_fields = len(rules), len(valid_tickets[0])
  check_eq(num_rules, num_fields)

  def is_compatible(rule, field_index):
    ranges = rules[rule]
    return all(
        any(range[0] <= ticket[field_index] <= range[1] for range in ranges)
        for ticket in valid_tickets
    )

  grid = np.empty((num_rules, num_fields), dtype=np.bool)
  # Computational bottleneck; could let rules.values() be array and use numba.
  for rule_index, rule in enumerate(rules):
    for field_index in range(num_fields):
      grid[rule_index, field_index] = is_compatible(rule, field_index)

  column_of_rule = {}
  row_sums = np.sum(grid, axis=1)
  while np.any(row_sums):
    row = np.nonzero(row_sums == 1)[0][0]
    col = np.nonzero(grid[row] == 1)[0][0]
    rule = list(rules)[row]
    column_of_rule[rule] = col
    row_sums -= grid[:, col]
    grid[:, col] = 0
  values = [my_ticket[column_of_rule[rule]] for rule in rules
            if rule.startswith('departure')]
  check_eq(len(values), 6)
  return np.prod(values)


puzzle.verify(2, process2)

(Time:  0.044)


## Day 17: Game of life in 3D and 4D

Given an initial 2D grid of cell states, simulate a cellular automaton on a higher-dimensional grid.  An inactive grid cell becomes active if exactly 3 of its neighbors is active, and an active grid cell becomes inactive unless 2 or 3 of its neighbors is active.  (Each cell has 26 neighbors in 3D and 80 neighbors in 4D.)

- Part 1: Run the cellular automaton in 3D for 6 generations and report the number of active cells.

- Part 2: Do the same in 4D.

In [85]:
puzzle = advent.puzzle(17)

For day 17, puzzle.input has 8 lines:
.......#
....#...
...###.#
#...###.
....##..
##.#..#.
###.#.#.
....#...
And the stored answers are: {1: 232, 2: 1620}


In [86]:
s1 = """
.#.
..#
###
"""

In [87]:
# Both of the solutions below work in arbitrary dimension!

In [88]:
# Slower code:
def process(s, num_cycles=6, dim=3):
  lines = s.strip('\n').splitlines()
  indices = {
      (0,) * (dim - 2) + (y, x)
      for y, line in enumerate(lines) for x, ch in enumerate(line)
      if ch == '#'
  }
  offsets = set(itertools.product((-1, 0, 1), repeat=dim)) - {(0,) * dim}

  def neighbors(index):
    for offset in offsets:
      yield tuple(map(sum, zip(index, offset)))

  def count_neighbors(index):
    return sum(neighbor in indices for neighbor in neighbors(index))

  for _ in range(num_cycles):
    survivors = {index for index in indices if 2 <= count_neighbors(index) <= 3}
    adjacents = {neighbor for index in indices for neighbor in neighbors(index)}
    births = {index for index in adjacents if count_neighbors(index) == 3}
    indices = survivors | births

  return len(indices)


check_eq(process(s1), 112)
puzzle.verify(1, process)

(Time:  0.146)


In [89]:
# Faster code:
def process(s, num_cycles=6, dim=3):
  lines = s.strip('\n').splitlines()
  indices = {
      (0,) * (dim - 2) + (y, x)
      for y, line in enumerate(lines) for x, ch in enumerate(line)
      if ch == '#'
  }
  offsets = set(itertools.product((-1, 0, 1), repeat=dim)) - {(0,) * dim}

  def neighbors(index):
    for offset in offsets:
      yield tuple(map(sum, zip(index, offset)))

  for _ in range(num_cycles):
    # Adapted from collections.Counter() algorithm in 
    # https://github.com/norvig/pytudes/blob/master/ipynb/Advent-2020.ipynb
    neighbor_counts = collections.Counter(
        itertools.chain.from_iterable(neighbors(index) for index in indices))
    indices = {index for index, count in neighbor_counts.items()
               if count == 3 or (count == 2 and index in indices)}

  return len(indices)


check_eq(process(s1), 112)
puzzle.verify(1, process)

(Time:  0.018)


Part 2

In [90]:
process2 = functools.partial(process, dim=4)
check_eq(process2(s1), 848)
puzzle.verify(2, process2)

(Time:  0.325)


In [91]:
def show_num_active_in_each_generation_for_2d_3d_4d():
  for dim in range(2, 5):
    print(dim, [process(puzzle.input, n, dim) for n in range(6)])

show_num_active_in_each_generation_for_2d_3d_4d()

2 [22, 17, 23, 24, 28, 27]
3 [22, 39, 50, 84, 140, 173]
4 [22, 105, 144, 652, 372, 1796]


## Day 18: Parsing math expression

Given a list of mathematical expressions with additions, multiplications, and parantheses, evaluate each expression and report the sum of resulting values.

- Part 1: Operators `+` and `*` have equal precedence and are applied left-to-right.

- Part 2: Operator `+` has higher precedence than `*` (which is unusual).

In [92]:
puzzle = advent.puzzle(18)

For day 18, puzzle.input has 380 lines:
7 + (9 * 8 + 5 + 5 * (3 * 4 * 7 + 6 * 4)) * ((3 * 6 + 3 * 4 * 7 * 4) + 4 * 3 * 5 + 5 * (5 * 6 + 7)) * 2 + 6 * 4
9 * 4 * ((9 * 8 + 9 + 2 + 9) + 2 * 9 + 2 + 2) * 5 * 6
3 * ((9 * 3 * 8 * 6 * 6 * 7) + 8) * 2 * 9 + 4 * 8
(3 + 4 + 4 * 4 + 9) + (7 + 6 + 2 * 8) * 9 + 7 * 8
9 + 6 + 6 + 9 * 9 * 5
(3 * 5 + (4 + 3 * 7 * 8) + 7 + 8 + 2) * 8 * (3 * 4 * 8) * ((4 + 5 * 2) * 7)
6 + (5 + 8 * (6 * 8 * 9 + 9 + 8 * 9) * 4 + (5 * 9)) * 2 + 2
(6 + 8 + (6 * 6 + 2) * 6 * (7 + 7 + 8 + 9)) * 3 + 8 + (6 * (8 + 8 + 9)) + 2
 ...
5 * 8 + (7 + 2 + 6) * (8 + 2 + 8 + 4 + 9) + (5 * 6 * 6 * 3) + 8
6 + (7 * 4 + 4 * 7 + 3 * 7) + 9 + 5 * 7 * 5
3 + 2 + 8 + (4 + 9 * 4 * 2 + (9 * 9 + 9 + 5) * 5) + 6
3 * 9 * 4 * (5 * 4 * (8 + 3))
And the stored answers are: {1: 4940631886147, 2: 283582817678281}


In [93]:
# Slower, more readable version with many assertions:
def process(strings, part2=False):

  def evaluate_line(s):

    def parse_term(i):
      if s[i] == '(':
        value, i = parse_sequence(i + 1)
        check_eq(s[i], ')')
        return value, i + 1
      if s[i].isdigit():
        value = int(s[i])
        return value, i + 1
      raise AssertionError

    def parse_sequence(i):
      value, i = parse_term(i)
      while i < len(s) and s[i] != ')':
        check_eq(s[i], ' ')
        check_eq(s[i + 2], ' ')
        if s[i + 1] == '+':
          value2, i = parse_term(i + 3)
          value += value2
        elif s[i + 1] == '*':
          value2, i = (parse_sequence if part2 else parse_term)(i + 3)
          value *= value2
        else:
          raise AssertionError
      return value, i

    value, i = parse_sequence(0)
    check_eq(i, len(s))
    return value

  return sum(evaluate_line(s) for s in strings.splitlines())

puzzle.verify(1, process)

process2 = functools.partial(process, part2=True)
puzzle.verify(2, process2)

(Time:  0.053)
(Time:  0.051)


In [94]:
# Compact faster version:
def process(strings, part2=False):

  def evaluate_line(s):

    def eval_term(i):
      return eval_seq(i + 1, eat=1) if s[i] == '(' else (int(s[i]), i + 1)

    def eval_seq(i, eat=0):
      value, i = eval_term(i)
      while i < len(s) and s[i] != ')':
        is_mul = s[i + 1] == '*'
        value2, i = (eval_seq if part2 and is_mul else eval_term)(i + 3)
        value = value * value2 if is_mul else value + value2
      return value, i + eat

    return eval_seq(0)[0]

  return sum(evaluate_line(s) for s in strings.strip('\n').splitlines())


check_eq(process('1 + 2 * 3 + 4 * 5 + 6'), 71)
check_eq(process('1 + (2 * 3) + (4 * (5 + 6))'), 51)
check_eq(process('2 * 3 + (4 * 5)'), 26)
check_eq(process('5 + (8 * 3 + 9 + 3 * 4 * 3)'), 437)
check_eq(process('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))'), 12240)
check_eq(process('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2'), 13632)

puzzle.verify(1, process)

(Time:  0.004)


In [95]:
process2 = functools.partial(process, part2=True)

check_eq(process2('1 + 2 * 3 + 4 * 5 + 6'), 231)
check_eq(process2('1 + (2 * 3) + (4 * (5 + 6))'), 51)
check_eq(process2('2 * 3 + (4 * 5)'), 46)
check_eq(process2('5 + (8 * 3 + 9 + 3 * 4 * 3)'), 1445)
check_eq(process2('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))'), 669060)
check_eq(process2('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2'), 23340)

puzzle.verify(2, process2)

(Time:  0.006)


## Day 19: Message grammar

Given a set of context-free grammar rules and a set of words, report the number of words that are expressible in the grammar.

- Part 1: Use the initial grammar rules, which do not have cycles.

- Part 2: Replace two of the grammar rules, thereby introducing cycles.

In [96]:
puzzle = advent.puzzle(19)

For day 19, puzzle.input has 583 lines:
26: 97 126 | 123 57
122: 84 97 | 92 123
82: 97 138 | 123 130
80: 131 97
3: 123 107 | 97 66
101: 1 123 | 95 97
10: 97 138 | 123 107
83: 123 27 | 97 12
 ...
abaabababbbbbabababbabbaaaabbbba
baaaababbbabbbbaaababbab
bbbabbabaabbaabbabbaaabb
abaaaababaabbbbabbbbbbbbaaaaaaaababaaaabbbaaaaaabbbbaaabbbbababb
And the stored answers are: {1: 198, 2: 372}


In [97]:
#@title
print('Let s1, s2 be test inputs.')

s1 = """
0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb
"""

s2 = """
42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
"""

Let s1, s2 be test inputs.


In [98]:
# Compact code:
def process(s, part2=False):
  section1, section2 = s.strip('\n').split('\n\n')
  rules = dict(line.split(': ') for line in section1.splitlines())
  if part2:
    rules.update({'8': '42 | 42 8', '11': '42 31 | 42 11 31'})

  def valid_expansion(symbols, text):
    if not symbols or not text:
      return not symbols and not text
    expansions = rules[symbols[0]]
    if expansions[0] == '"':
      return expansions[1] == text[0] and valid_expansion(symbols[1:], text[1:])
    return any(valid_expansion(expansion.split() + symbols[1:], text)
               for expansion in expansions.split(' | '))

  return sum(valid_expansion(['0'], text) for text in section2.splitlines())


check_eq(process(s1), 2)
puzzle.verify(1, process)

process2 = functools.partial(process, part2=True)
check_eq(process2(s2), 12)
puzzle.verify(2, process2)

(Time:  0.085)
(Time:  0.410)


In [99]:
# Faster code:
def process(s, part2=False):
  section1, section2 = s.strip('\n').split('\n\n')
  rules = {
      int(symbol): (rhs[1] if rhs[0] == '"' else
                    tuple(tuple(map(int, s.split())) for s in rhs.split(' | ')))
      for line in section1.splitlines()
      for symbol, rhs in (line.split(': '),)
  }
  if part2:
    # We are fortunate that in both modified rules, there is a symbol to the
    # left of the recursion, which forces the consumption of a character.
    rules.update({8: ((42,), (42, 8)), 11: ((42, 31), (42, 11, 31))})

  def valid_expansion(symbols, text):
    while True:
      if not symbols:
        return not text
      if not text:
        return False
      expansions = rules[symbols[0]]
      if isinstance(expansions, str):
        if expansions != text[0]:
          return False
        symbols, text = symbols[1:], text[1:]
        continue
      return any(valid_expansion(expansion + symbols[1:], text)
                for expansion in expansions)

  return sum(valid_expansion((0,), text) for text in section2.splitlines())


check_eq(process(s1), 2)
puzzle.verify(1, process)

process2 = functools.partial(process, part2=True)
check_eq(process2(s2), 12)
puzzle.verify(2, process2)

(Time:  0.066)
(Time:  0.275)


## Day 20: Assemble rotated tiles

Given a set of numbered 2D image tiles with binary values, assemble these tiles using rotations and flips into a 2D grid such that adjacent tiles have matching boundary values.

- Part 1: Return the product of the tile numbers for the 4 tiles that lie at the corners of the assembled 2D grid.

- Part 2: Given a 2D mask ("dragon"), find all possible locations for this mask (including rotations and flips) such that all active values in the mask are also active in the assembled grid.  Report the number of active grid nodes which are not covered by any found mask instance.

In [100]:
puzzle = advent.puzzle(20)

For day 20, puzzle.input has 1727 lines:
Tile 2789:
.#..#...##
##...#....
#.#.##..#.
.#.##.##.#
....#....#
#.#......#
.....#....
 ...
..#......#
#......#.#
..#.....#.
.#.#.##.##
And the stored answers are: {1: 18449208814679, 2: 1559}


In [101]:
#@title

print('Let s1 be a test input.')

s1 = """
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...
"""

Let s1 be a test input.


In [102]:
def process(s, part2=False):

  def to_grid(s):
    return np.array(list(map(list, s.strip('\n').splitlines())))

  tiles = {int(t[5:9]): to_grid(t[11:]) for t in s.strip('\n').split('\n\n')}
  n = int(len(tiles)**0.5)  # math.isqrt() in version 3.8
  rotations = tuple(range(8))  # 4 proper rotations * 2 flips

  def rotate(tile, rotation):
    return np.rot90(tile if rotation < 4 else tile[::-1], rotation % 4)

  # List of (index, rotation) for up to 2 tiles whose top row matches the key.
  edge_list = collections.defaultdict(list)
  for index, tile in tiles.items():
    for rotation in rotations:
      edge_list[tuple(rotate(tile, rotation)[0])].append((index, rotation))

  def is_corner(tile):
    # A corner tile has 2 unique edges.
    return sum(len(edge_list[tuple(rotate(tile, rotation)[0])]) == 1
               for rotation in rotations[:4]) == 2

  corners = [index for index, tile in tiles.items() if is_corner(tile)]
  assert len(corners) == 4  # Exactly four tiles must be at the corners.
  if not part2:
    return np.prod(corners)

  # Place a first corner in the grid upper-left and determine its rotation.
  index = corners[0]
  tile = tiles[index]
  rotation, _ = (rotation for rotation in rotations  # 2 solutions due to flip
                 if (len(edge_list[tuple(rotate(tile, rotation)[0])]) == 1 and
                     len(edge_list[tuple(rotate(tile, rotation)[:, 0])]) == 1))
  layout = np.empty((n, n), dtype=object)
  layout[0, 0] = (index, rotation)

  def find(not_index, rot, desired):
    "Returns tile (index, rotation) with top row matching desired after rot."
    l = edge_list[tuple(desired)]
    index, rotation = next(((index, rotation) for index, rotation in l
                            if index != not_index))
    return index, (rotation + rot) % 4 + rotation // 4 * 4

  for y, x in np.ndindex(n, n):
    if x > 0:
      left_index, left_rotation = layout[y, x - 1]
      desired_left = rotate(tiles[left_index], left_rotation)[::-1, -1]
      layout[y, x] = find(left_index, 1, desired_left)
    elif y > 0:
      top_index, top_rotation = layout[y - 1, x]
      desired_top = rotate(tiles[top_index], top_rotation)[-1, :]
      layout[y, x] = find(top_index, 0, desired_top)

  def block(y, x):
    index, rotation = layout[y, x]
    return rotate(tiles[index], rotation)[1:-1, 1:-1]

  grid = np.block([[block(y, x) for x in range(n)] for y in range(n)])

  pattern = '                  # #    ##    ##    ### #  #  #  #  #  #   '
  pattern = np.array(list(pattern)).reshape(3, 20) == '#'
  not_pattern = ~pattern

  for rotation in rotations:
    gridview = rotate(grid, rotation)
    for y, x in np.ndindex(*(np.array(gridview.shape) - pattern.shape)):
      subgrid = gridview[y:y + pattern.shape[0], x:x + pattern.shape[1]]
      # Computational bottleneck in part2.
      if np.all(not_pattern | (subgrid != '.')):
        subgrid[pattern] = 'O'

  return np.count_nonzero(grid == '#')


check_eq(process(s1), 20899048083289)
puzzle.verify(1, process)

process2 = functools.partial(process, part2=True)
check_eq(process2(s1), 273)
puzzle.verify(2, process2)

(Time:  0.027)
(Time:  0.517)


## Day 21: Allergens in ingredients

Given a set of foods, each containing a list of ingredients and a set of allergens contained in the ingredients, determine the ingredient corresponding to each allergen.  Each allergen is contained in at most one ingredient.  Some foods may fail to list all allergens.

- Part 1: For each food, count the number of ingredients which cannot possibly contain allergens, and report the sum.

- Part 2: Report the comma-separated list of ingredients which contain allergens, sorted in ascending order by the allergen name.

In [103]:
puzzle = advent.puzzle(21)

For day 21, puzzle.input has 42 lines:
xfvxzl nfcrh kscdhn khnqq jkvcbf jhlvrg chc fk zbh hgrptqb zrvtg tdhv jhqpdf jvj ... mxbbst (contains shellfish, sesame)
vxzpfp shlg zrvtg ghq vzn ppqs mqlnbjq cltpv pvxpgp psbnqz nqcmm sqjcsxjq xftxgf ... cbhfpzp khnqq dhqbf (contains fish)
fzhmtds zbclt pbln qtgv bnl xqcb vxzpfp jkvcbf nnst tplrt xknj hzmvzrr xfvxzl gk ...  (contains sesame, fish, shellfish)
ghq vfqh vtdbl bkmgx bxlc sbhdb xfvxzl lvdcdg nsktq hrjrm dbr prrfm dbqbv jbcjk  ... gpgql dpkkzdgs cjds (contains nuts)
xknj bbbzcmh tzrp xkqgc ktfvb tplrt dqz ntq jbcjk pcnmbt nqcmm tlgchbh bxfvt nfs ... hlvrg fhznhmx (contains dairy, soy)
strd dpdgc cjdnjf flmssjn xfpc mmdfrfcp gpp tqjg znkr mxbbst bbbzcmh jbvrzn cdsl ... mx xknh schkf (contains soy, wheat)
gkgzn bndz dpkkzdgs nv nhjbd mqlnbjq khnqq ljpzx cfbgb xknj ddmg mkpmkx tgcvz nn ... trd tdhv fk tdsvx (contains sesame)
gnfjcpc ddmg tpbtpd cltpv xfvxzl txr rmxzzf tgcvz jsdgn pgnk jrpx vbvfr vzn gbpl ... c dvzqx hcms zrvtg (contains 

In [104]:
s1 = """
mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)
"""

In [105]:
def process(s, part2=False):

  def foods():
    for line in s.strip('\n').splitlines():
      ingredients, allergens = line[:-1].split(' (contains ')
      yield set(ingredients.split()), allergens.split(', ')

  possibles_for_allergen = {}
  for ingredients, allergens in foods():
    for allergen in allergens:
      if allergen not in possibles_for_allergen:
        possibles_for_allergen[allergen] = ingredients.copy()
      else:
        possibles_for_allergen[allergen] &= ingredients

  allergen_ingredient = {}
  while possibles_for_allergen:
    has_single_ingredient = lambda item: len(item[1]) == 1
    allergen, ingredients = next(
        filter(has_single_ingredient, possibles_for_allergen.items()))
    ingredient = ingredients.pop()
    allergen_ingredient[allergen] = ingredient
    for ingredients2 in possibles_for_allergen.values():
      ingredients2.discard(ingredient)
    del possibles_for_allergen[allergen]

  if not part2:
    allergen_ingredients = set(list(allergen_ingredient.values()))
    return sum(
        len(ingredients - allergen_ingredients) for ingredients, _ in foods())

  return ','.join(ingredient
                  for _, ingredient in sorted(allergen_ingredient.items()))


check_eq(process(s1), 5)
puzzle.verify(1, process)

process2 = functools.partial(process, part2=True)
check_eq(process2(s1), 'mxmxvkd,sqjhc,fvjkl')
puzzle.verify(2, process2)

(Time:  0.001)
(Time:  0.001)


## Day 22: Card game

Given two hands of integer cards, simulate a card game of combat.  At each turn, the player with the largest front card places the two cards (higher card first) at the back of their hand.  A player that owns all cards wins.

- Part 1: In the winner's hand, multiply each card value by its position (1 being rearmost), and report the sum of the result.

- Part 2: Add extra rules.  If both players have at least as many cards remaining in their deck as their top card, the winner of the round is determined by playing a new game of Recursive Combat.  See the [website](https://adventofcode.com/2020/day/22) for details.

In [106]:
puzzle = advent.puzzle(22)

For day 22, puzzle.input has 53 lines:
Player 1:
15
31
26
4
36
30
43
 ...
37
5
1
27
And the stored answers are: {1: 32199, 2: 33780}


In [107]:
s1 = """
Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10
"""

In [108]:
def process1(s):
  hands = [collections.deque(map(int, s2.splitlines()[1:]))
           for s2 in s.strip('\n').split('\n\n')]

  while hands[0] and hands[1]:
    cards = hands[0].popleft(), hands[1].popleft()
    winner = 0 if cards[0] > cards[1] else 1
    hands[winner].extend([cards[winner], cards[1 - winner]])

  return sum((i + 1) * card for i, card in enumerate(reversed(hands[winner])))


check_eq(process1(s1), 306)
puzzle.verify(1, process1)

(Time:  0.000)


Part 2

In [109]:
# Slower code using deque:
def process2(s):
  hands = [collections.deque(map(int, s2.splitlines()[1:]))
           for s2 in s.strip('\n').split('\n\n')]

  def combat(hands):  # Returns (hands, winner)
    visited = set()
    while hands[0] and hands[1]:
      state = (*hands[0], -1, *hands[1])
      if state in visited:
        return hands, 0  # player 1 is winner
      visited.add(state)
      cards = hands[0].popleft(), hands[1].popleft()
      if len(hands[0]) >= cards[0] and len(hands[1]) >= cards[1]:
        _, winner = combat([collections.deque(list(hands[0])[:cards[0]]),
                            collections.deque(list(hands[1])[:cards[1]])])
      else:
        winner = 0 if cards[0] > cards[1] else 1
      hands[winner].extend([cards[winner], cards[1 - winner]])

    return hands, 0 if hands[0] else 1

  hands, winner = combat(hands)
  return sum((i + 1) * card for i, card in enumerate(reversed(hands[winner])))


check_eq(process2(s1), 291)

In [110]:
# Faster code using tuples:
def process2(s):
  hand0, hand1 = (tuple(map(int, s2.splitlines()[1:]))
                  for s2 in s.strip('\n').split('\n\n'))

  def combat(hand0, hand1):
    visited = set()
    while True:
      state = (hand0, hand1)
      if state in visited:
        return (0,), ()  # player 1 is winner
      visited.add(state)
      card0 = hand0[0]
      card1 = hand1[0]
      recurse = len(hand0) > card0 and len(hand1) > card1
      if (combat(hand0[1:card0 + 1], hand1[1:card1 + 1])[0] if recurse
          else card0 > card1):
        hand0, hand1 = hand0[1:] + (card0, card1), hand1[1:]
        if not hand1:
          break
      else:
        hand0, hand1 = hand0[1:], hand1[1:] + (card1, card0)
        if not hand0:
          break
    return hand0, hand1

  hand0, hand1 = combat(hand0, hand1)
  return sum((i + 1) * card for i, card in enumerate(reversed(hand0 + hand1)))


check_eq(process2(s1), 291)
puzzle.verify(2, process2)

(Time:  1.000)


In [111]:
# %timeit -r8 process2(puzzle.input)

In [112]:
# Also tried:
# - Created version which caches outcomes of states.  It is 1.7x slower,
#    likely because there are too many possible states.
# - Used separate recurse() function returning winner and no hands; no change.

In [113]:
# %prun process2(puzzle.input)  # No obvious opportunity for optimization.

## Day 23: Cups in a circle

Given a circular list of cups (labeled 1-9), apply a sequence of moves.
For each move, extract the 3 cups to the right of the current cup (initially first in the input), and reinsert the 3 cups to the right of the cup whose label has value 1 less than the current cup.  Advance the current cup to the right.

When finding the cup whose label has value 1 less than the current cup, skip over any that are in the 3 extracted cups, and when reaching value 0, wraparound back to the highest value.

- Part 1: After 100 moves, what are the cup labels (as a string of 8 digits) to the right of cup 1?

- Part 2: Extend the input circular list to contain a total of 1_000_000 cups, labeled 10 .. 1_000_000.  After 10_000_000 moves, determine the two cups to the right of cup 1, and report the product of their labels.

In [114]:
puzzle = advent.puzzle(23)

For day 23, puzzle.input has 1 lines:
872495136
And the stored answers are: {1: '27865934', 2: 170836011000}


In [115]:
s1 = '389125467'

In [116]:
# Code using list:
def process(s, num_moves=100):
  l = list(map(int, s.strip()))

  for _ in range(num_moves):
    (current, *extracted3) = l[:4]
    destination = current - 1 if current > 1 else len(l)
    while destination in extracted3:
      destination = destination - 1 if destination > 1 else len(l)
    dest = l.index(destination)  # bottleneck in part 2!
    l = l[4:dest + 1] + extracted3 + l[dest + 1:] + [current]

  i = l.index(1)
  l = l[i + 1:] + l[:i]
  return ''.join(map(str, l))


check_eq(process(s1, num_moves=10), '92658374')
check_eq(process(s1), '67384529')

In [117]:
# Code using deque:
def process(s, num_moves=100):
  d = collections.deque(map(int, s.strip()))
  n = len(d)

  for _ in range(num_moves):
    current = next(iter(d))
    d.rotate(-1)  # move current to the end
    extracted3 = d.popleft(), d.popleft(), d.popleft()
    destination = current - 1 if current > 1 else n
    while destination in extracted3:
      destination = destination - 1 if destination > 1 else n
    dest = d.index(destination)  # bottleneck in part 2!
    d.rotate(-dest - 1)  # move destination to end
    d.extendleft(reversed(extracted3))
    d.rotate(dest + 1)  # move new current back to front

  d.rotate(-d.index(1))  # move '1' to front
  d.popleft()  # remove '1'
  return ''.join(map(str, d))


check_eq(process(s1, num_moves=10), '92658374')
check_eq(process(s1), '67384529')

In [118]:
# Code keeping track of next cup label for each cup label:
def process(s, max_num=0, num_moves=100):
  dtype = np.int32 if using_numba else np.int64
  l = np.array(list(map(int, s.strip())), dtype)
  next_cup = np.empty(1 + max(len(l), max_num), dtype=l.dtype)

  @numba_njit(cache=True)  # 0.25 s to jit on first invocation
  def func(l, max_num, num_moves, next_cup):
    n = len(next_cup) - 1
    current = l[0]
    for i in range(len(l) - 1):
      next_cup[l[i]] = l[i + 1]
    if max_num:
      next_cup[l[-1]] = len(l) + 1
      for i in range(len(l) + 1, max_num):
        next_cup[i] = i + 1
      next_cup[max_num] = l[0]
    else:
      next_cup[l[-1]] = l[0]

    for _ in range(num_moves):
      extracted1 = next_cup[current]
      extracted2 = next_cup[extracted1]
      extracted3 = next_cup[extracted2]
      destination = current - 1 if current > 1 else n
      while destination in (extracted1, extracted2, extracted3):
        destination = destination - 1 if destination > 1 else n
      next_cup[current] = next_cup[extracted3]
      next_cup[extracted3] = next_cup[destination]
      next_cup[destination] = extracted1
      current = next_cup[current]

    return current

  current = func(l, max_num, num_moves, next_cup)
  if max_num:
    cup1 = next_cup[1]
    cup2 = next_cup[cup1]
    return int(cup1) * int(cup2)
  else:
    l = [next_cup[1]]
    while next_cup[l[-1]] != 1:
      l.append(next_cup[l[-1]])
    return ''.join(map(str, l))


check_eq(process(s1, num_moves=10), '92658374')
check_eq(process(s1), '67384529')

puzzle.verify(1, process)

(Time:  0.004)


Part 2

In [119]:
# ~20 s without numba; ~0.5 s with numba
process2 = functools.partial(process, num_moves=10_000_000, max_num=1_000_000)

if using_numba:
  check_eq(process2(s1), 149245887792)

puzzle.verify(2, process2)

(Time:  0.392)


In [120]:
# %timeit -r8 process2(puzzle.input)

In [121]:
# %prun process2(puzzle.input)  # crashes runtime, likely due to numba function

## Day 24: Hexagonal tiles

For each input line, travel over a hexagonal grid using the substrings `e`, `w`, `ne`, `nw`, `se`, and `sw`, and flip the state of the reached grid cell.  (Initially all grid cells are inactive.)

- Part 1: Report the number of active cells in the resulting hexagonal grid.

- Part 2: Simulate a cellular automaton over the grid from part 1, where an inactive cell becomes active if it has exactly 2 active neighbors,
and an active cell becomes inactive if it has 0 or more than 2 active neighbors.  Report the number of active cells after 100 steps.

In [122]:
puzzle = advent.puzzle(24)

For day 24, puzzle.input has 347 lines:
enwwsenweswweseseswnenewwswseswenw
nwnwneswwnenwnwnwenenwwnesenenwsenenee
eneswwenwneeswneenenw
swwswswwwswswsweeswww
swewnwneeewnewwwswwnwwswswnwww
seswnwenesenwsenenwwenwsenwswnwseswnw
nwswwenwwwswwnwnewswnwnwwwswnee
seswseswseseseseeseseseswnwsesw
 ...
nwnwnwnwnewnwwwnwnwnenenwsenwsesenwnww
nwwswneseseswsenweswswswneswwswwswswsw
neenwsenenenenenenwswswnenenwsenenene
newnwnwnwnwwnweswwewsewenwwnwnwnwnw
And the stored answers are: {1: 287, 2: 3636}


In [123]:
#@title
print('Let s1 be a test input.')

s1 = """
sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
"""

Let s1 be a test input.


In [124]:
def process(s, num_days=0):
  offsets = dict(e=(0, 1), w=(0, -1), sw=(1, 0), se=(1, 1),
                 nw=(-1, -1), ne=(-1, 0))
  tuple_offsets = tuple(offsets.values())  # slightly faster
  indices = set()
  for line in s.strip('\n').splitlines():
    y, x = 0, 0
    while line:
      for prefix, (dy, dx) in offsets.items():
        if line.startswith(prefix):
          y, x = y + dy, x + dx
          line = line[len(prefix):]
    indices ^= {(y, x)}

  for _ in range(num_days):
    neighbor_counts = collections.Counter(
        (y + dy, x + dx) for y, x in indices for dy, dx in tuple_offsets)
    indices = {index for index, count in neighbor_counts.items()
               if count == 2 or (count == 1 and index in indices)}

  return len(indices)


check_eq(process(s1), 10)
puzzle.verify(1, process)

(Time:  0.006)


Part 2

In [125]:
check_eq(process(s1, num_days=10), 37)

process2 = functools.partial(process, num_days=100)
check_eq(process2(s1), 2208)
puzzle.verify(2, process2)

(Time:  0.407)


## Day 25: Public-key encryption

Given a card public key and a door public key, find the (private) card loop size and door loop size that were used to generate these public keys.  Then, find the common encryption key that the card and door use to communicate.

- Part 1: Report the encryption key.

- Part 2: There is no part 2 for this last day of the Advent.

In [126]:
puzzle = advent.puzzle(25)

For day 25, puzzle.input has 2 lines:
10212254
12577395
And the stored answers are: {1: 290487, 2: 'no_part2_on_last_day'}


In [127]:
s1 = """
5764801
17807724
"""

In [128]:
# Slow code
def process(s, base=7, mod=20201227):
  card_public_key, door_public_key = map(int, s.strip('\n').splitlines())

  # Slow trial-multiplication; https://en.wikipedia.org/wiki/Discrete_logarithm
  def untransform(base, value):
    x = 1
    loop_size = 0
    while x != value:
      loop_size += 1
      x = (x * base) % mod
    return loop_size

  # Slow O(n) version.
  def transform(value, loop_size):
    x = 1
    for _ in range(loop_size):
      x = (x * value) % mod
    return x

  card_loop_size = untransform(base, card_public_key)
  door_loop_size = untransform(base, door_public_key)
  assert transform(base, card_loop_size) == card_public_key
  assert transform(base, door_loop_size) == door_public_key

  encryption_key1 = transform(door_public_key, card_loop_size)
  encryption_key2 = transform(card_public_key, door_loop_size)
  assert encryption_key1 == encryption_key2
  return encryption_key1


check_eq(process(s1), 14897079)

In [129]:
# Fast code

def pow_mod(base: int, exponent: int, mod: int) -> int:
  """Returns 'base**exponent % mod' using square-multiply algorithm."""
  x = 1
  while exponent > 0:
    if exponent % 2 == 1:
      x = (x * base) % mod
    base = (base * base) % mod
    exponent //= 2
  return x


def log_mod(base: int, value: int, mod: int) -> Optional[int]:
  """Returns exponent for 'base**exponent % mod == value'."""
  # Using https://en.wikipedia.org/wiki/Baby-step_giant-step
  m = int(math.ceil(math.sqrt(mod)))
  table = {}
  e = 1
  for i in range(m):
    table[e] = i
    e = (e * base) % mod
  factor = pow_mod(base, mod - m - 1, mod)
  e = value
  for i in range(m):
    if e in table:
      return i * m + table[e]
    e = (e * factor) % mod
  return None


def process(s, base=7, mod=20201227):
  card_public_key, door_public_key = map(int, s.strip('\n').splitlines())
  card_loop_size = log_mod(base, card_public_key, mod)
  return pow_mod(door_public_key, card_loop_size, mod)


check_eq(process(s1), 14897079)
puzzle.verify(1, process)

(Time:  0.001)


Part 2

In [130]:
puzzle.verify(2, lambda x: 'no_part2_on_last_day')

(Time:  0.000)


## Timings

In [131]:
advent.show_times()

day_1    part_1: 0.000   part_2: 0.002
day_2    part_1: 0.005   part_2: 0.003
day_3    part_1: 0.002   part_2: 0.005
day_4    part_1: 0.001   part_2: 0.005
day_5    part_1: 0.001   part_2: 0.001
day_6    part_1: 0.001   part_2: 0.002
day_7    part_1: 0.003   part_2: 0.003
day_8    part_1: 0.000   part_2: 0.010
day_9    part_1: 0.002   part_2: 0.002
day_10   part_1: 0.000   part_2: 0.000
day_11   part_1: 0.042   part_2: 0.047
day_12   part_1: 0.000   part_2: 0.000
day_13   part_1: 0.000   part_2: 0.000
day_14   part_1: 0.002   part_2: 0.022
day_15   part_1: 0.001   part_2: 0.786
day_16   part_1: 0.005   part_2: 0.044
day_17   part_1: 0.018   part_2: 0.325
day_18   part_1: 0.004   part_2: 0.006
day_19   part_1: 0.066   part_2: 0.275
day_20   part_1: 0.027   part_2: 0.517
day_21   part_1: 0.001   part_2: 0.001
day_22   part_1: 0.000   part_2: 1.000
day_23   part_1: 0.004   part_2: 0.392
day_24   part_1: 0.006   part_2: 0.407
day_25   part_1: 0.001   part_2: 0.000
Total time:  4.047


In [132]:
if 0:  # Compute min execution times over several calls.
  advent.show_times(recompute=True, repeat=3)

In [133]:
if 0:  # Look for unwanted pollution of namespace.
  print(textwrap.fill(' '.join(var for var, value in globals().items() if (
      not var.startswith('_') and not repr(value).startswith(
          ('<module', '<class', 'typing.', 'functools.partial('))))))

In [134]:
if 0:  # Print the answers to update an external file
  print(json.dumps(advent.answers, sort_keys=True, indent=2))

# End